In [1]:
import pandas as pd
import numpy as np
import math
import torch
import matplotlib.pyplot as plt
from peft import LoraConfig, get_peft_model
from torch import nn
from photutils import EllipticalAperture
from photutils.aperture import aperture_photometry
from torch.utils.data.sampler import Sampler
from collections import defaultdict
import random

/tmp/ipykernel_68/2486450629.py:8: DeprecationWarning: `photutils.EllipticalAperture` is a deprecated alias for `photutils.aperture.EllipticalAperture` and will be removed in the future. Instead, please use `from photutils.aperture import EllipticalAperture` to silence this warning.
  from photutils import EllipticalAperture


In [2]:
df_train = pd.read_csv('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/df_train.csv')
df_val = pd.read_csv('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/df_val.csv')
df_train.drop(columns=['Unnamed: 0'], inplace=True)
df_val.drop(columns=['Unnamed: 0'], inplace=True)

stamp_train = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/stamp_train.npy')
stamp_val = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/stamp_val.npy')
mask_train = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_train.npy')
mask_val = np.load('/data/aai/scratch/jchan/denoise/PAUS/output_save/modify_pn2v/mask_val.npy')

In [3]:
df_train

,ref_id,I_auto,zp,aperture_x,aperture_y,aperture_theta,aperture_a,aperture_b,path
0,16634,21.424,4.309012,144.01718,3797.99580,-65.8252,1.009526,1.009526,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
1,16634,21.424,4.558836,1852.23990,3713.31880,-65.8252,1.089713,1.089713,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
2,16634,21.424,4.418699,1893.60950,3706.59130,-65.8252,0.948528,0.948528,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
3,16634,21.424,4.390103,1957.39890,3794.82860,-65.8252,0.942862,0.942862,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
4,16634,21.424,4.730091,1843.31860,3759.81790,-65.8252,1.012759,1.012759,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
...,...,...,...,...,...,...,...,...,...
18544,95442,20.170,4.536117,1889.10380,458.43564,66.1383,1.200518,1.178964,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
18545,95442,20.170,4.363345,97.61864,481.65180,66.1383,1.207213,1.185615,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
18546,95442,20.170,4.377721,1856.56500,462.17084,66.1383,1.277210,1.255294,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...
18547,95442,20.170,4.523878,1925.44240,528.73650,66.1383,1.061344,1.040054,/pnfs/pic.es/data/vo.paus.pic.es/paus/disk/arc...


In [4]:
#they are image and mask, don't need gradient
tensor_stamp_train = torch.from_numpy(stamp_train).float()
tensor_stamp_val = torch.from_numpy(stamp_val).float()
tensor_mask_train = torch.from_numpy(mask_train).float()
tensor_mask_val = torch.from_numpy(mask_val).float()

tensor_stamp_train = tensor_stamp_train.unsqueeze(1)
tensor_stamp_val = tensor_stamp_val.unsqueeze(1)
tensor_mask_train = tensor_mask_train.unsqueeze(1)
tensor_mask_val = tensor_mask_val.unsqueeze(1)

In [5]:
tensor_stamp_train.shape

torch.Size([18549, 1, 96, 96])

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18549 entries, 0 to 18548
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ref_id          18549 non-null  int64  
 1   I_auto          18549 non-null  float64
 2   zp              18549 non-null  float64
 3   aperture_x      18549 non-null  float64
 4   aperture_y      18549 non-null  float64
 5   aperture_theta  18549 non-null  float64
 6   aperture_a      18549 non-null  float64
 7   aperture_b      18549 non-null  float64
 8   path            18549 non-null  object 
dtypes: float64(7), int64(1), object(1)
memory usage: 1.3+ MB


In [7]:
features_df_train = df_train[['ref_id', 'zp', 'aperture_x', 'aperture_y', 'aperture_theta', 'aperture_a', 'aperture_b']].values
features_df_val = df_val[['ref_id', 'zp', 'aperture_y', 'aperture_theta', 'aperture_a', 'aperture_b']].values

In [8]:
features_df_train_tensor = torch.FloatTensor(features_df_train)
features_df_val_tensor = torch.FloatTensor(features_df_val)

In [9]:
import os
os.chdir('/data/aai/scratch/jchan/denoise/PAUS/dinggetest/simulation/pn2v/src/pn2v')
from core import prediction
from core import utils
from unet import UNet

device=utils.getDevice()

CUDA available? True


In [10]:
path='/data/aai/scratch/jchan/denoise/PAUS/dinggetest/simulation/model saved/'
model=torch.load(path+"/best_conv_N2V_PAUdm.net")

/tmp/ipykernel_68/4130286196.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model=torch.load(path+"/best_conv_N2V_PAUdm.net")


In [11]:
for param in model.parameters():
    param.requires_grad = False

In [12]:
#print(model)
target_list = [
    'conv_final',
    'down_convs.0.conv1',
    'down_convs.0.conv2',
    'down_convs.1.conv1',
    'down_convs.1.conv2',
    'down_convs.2.conv1',
    'down_convs.2.conv2',
    'up_convs.0.conv1',
    'up_convs.0.conv2',
    'up_convs.1.conv1',
    'up_convs.1.conv2'
]

In [13]:
config = LoraConfig(
    r=8,  #它控制了 LoRA 模块的“大小”或“复杂度”。r 越大，LoRA 模块的可训练参数就越多，理论上能学习更复杂的调整，但也会占用更多显存。r=8 或 16 是一个非常常见且高效的选择。
    lora_alpha=16, #LoRA 的输出会乘以一个缩放比例 alpha/r。这就像一个特殊的“学习率”或“平衡旋钮”。一个常见的经验法则是将 lora_alpha 设置为 r 的两倍（比如 r=8, alpha=16），这有助于稳定训练。
    target_modules=target_list,
    lora_dropout=0.1,#在 LoRA 模块中添加一个 Dropout 层，用于防止过拟合，这是一个标准的正则化技术。
)

lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 84,560 || all params: 1,761,938 || trainable%: 4.7993


In [14]:
#write the function to calculate the flux by myself
cutout_size = 48
def background_annulus_jiefeng(data, mask, aperture_x, aperture_y, r_in=30, r_out=45):
    
    masked_data = np.ma.array(data=data, mask=mask != 0)
    masked_data = masked_data.filled(fill_value=0)

    center = (aperture_x, aperture_y)
    annulus_apertures = CircularAnnulus(center, r_in=r_in, r_out=r_out)
    masks = annulus_apertures.to_mask(method='center')

    cutout_data = masks.cutout(masked_data)

    clip_annulus_array = sigma_clip(cutout_data[cutout_data != 0], sigma=3, maxiters=2)

    background_annulus = np.ma.mean(clip_annulus_array)
    #we use median here, in the dataset they use mean
    #background_annulus = np.ma.median(clip_annulus_array)
    return background_annulus

def flux_elliptical_jiefeng(image, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):

    image_shape = (cutout_size*2,cutout_size*2)
    PIXEL_SCALE = 0.263
    theta = -aperture_theta * np.pi / 180.
    a = aperture_a / PIXEL_SCALE
    b = aperture_b / PIXEL_SCALE

    center = (aperture_x, aperture_y)
    source_aperture = EllipticalAperture(center, a, b, theta)
    mask_object = source_aperture.to_mask(method='exact')
    mask_image_photutils_fractional = mask_object.to_image(shape=image_shape)
    
    xmask = mask != 0
    image_good = image * (1 - xmask)
    
    raw_flux = np.sum(image_good * mask_image_photutils_fractional)#calculate by myself

    background = background_annulus_jiefeng(image, mask, aperture_x, aperture_y)
    gal_flux = raw_flux - source_aperture.area * background
    
    return gal_flux

In [18]:
class PairedBatchSampler(Sampler):
    """
    choose P ref_ids, and choose 2 from each ref_id.
    
    P = 8, K = 2 , batch_size = 16
    batch_indices = [idx_A1, idx_G1, ... idx_F1,   idx_A2, idx_G2, ... idx_F2]
                  
    - ref_ids (list)
    - P (int): number of the pair
    """
    def __init__(self, ref_ids, P):
        super(PairedBatchSampler, self).__init__()
        
        if P <= 0:
            raise ValueError("P must be > 0")

        self.P = P
        self.K_is_fixed_at = 2
        self.batch_size = P * self.K_is_fixed_at
        
        print("constructing PairedBatchSampler ...")
        grouped_indices = defaultdict(list)
        for i, ref_id in enumerate(ref_ids):
            grouped_indices[ref_id].append(i)
        
        print(f"creating 'chunks' (size K={self.K_is_fixed_at})...")
        self.all_chunks = []
        for ref_id, indices in grouped_indices.items():
            if len(indices) >= self.K_is_fixed_at:#Acutally I have already selected the ref_ids
                
                random.shuffle(indices) #make it random
                
                # divide the ref_id. e.g. floor(13 / 4) = 3. we build 3 chunks。
                num_chunks_for_this_id = len(indices) // self.K_is_fixed_at
                
                for i in range(num_chunks_for_this_id):
                    chunk = indices[i * self.K_is_fixed_at : (i + 1) * self.K_is_fixed_at]
                    self.all_chunks.append(chunk)
        
        print(f"Already created {len(self.all_chunks)} 'K-chunks'.")
        
        if len(self.all_chunks) < P:
            raise ValueError(f"ref_ids are less than P={P} 'K-chunks'. Use a smaller K or P")
            
        self.num_batches = len(self.all_chunks) // P

    def __iter__(self):
        """
        PyTorch DataLoader before each epochs.
        """
        random.shuffle(self.all_chunks)
        #ref_ids_shuffled = random.sample(self.valid_ref_ids, len(self.valid_ref_ids))
        
        for i in range(self.num_batches):
            batch_part1_indices = []
            batch_part2_indices = []
            
            # 3. take P chunkscustom_sampler.py
            p_chunks = self.all_chunks[i * self.P : (i + 1) * self.P]
            
            for chunk in p_chunks:
                
                batch_part1_indices.append(chunk[0])
                batch_part2_indices.append(chunk[1])
            
            final_batch_indices = batch_part1_indices + batch_part2_indices
            
            # P + P = 16
            yield final_batch_indices

    def __len__(self):
        return self.num_batches

In [19]:
print(tensor_stamp_train.shape)
print(tensor_mask_train.shape)
print(features_df_train_tensor.shape)

torch.Size([18549, 1, 96, 96])
torch.Size([18549, 1, 96, 96])
torch.Size([18549, 7])


In [42]:
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(tensor_stamp_train, tensor_mask_train, features_df_train_tensor)

traindataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [43]:
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=1e-4)

In [8]:
def loss_unbias(label, output, mask, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):

    gal_flux_label = flux_elliptical_jiefeng(label, mask, aperture_x, aperture_y,
                                   aperture_theta, aperture_a, aperture_b)
    gal_flux_output = flux_elliptical_jiefeng(output, mask, aperture_x, aperture_y,
                                   aperture_theta, aperture_a, aperture_b)

    loss_unbias = (gal_flux_label - gal_flux_output).abs().sum()
    
    return loss_unbias

def loss_df(output, mask, ref_id, zp, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):

    gal_flux_output = flux_elliptical_jiefeng(output, mask, aperture_x, aperture_y,
                                   aperture_theta, aperture_a, aperture_b)

    gal_output_calibrated = gal_flux_output * zp
    
    return loss_df

In [16]:
class PairedDifferenceLoss(nn.Module):
    """    
    - Batch Size = 16
    - (0 - 7) is [A, G, M, X, B, Q, T, F]。
    - (8-15) is also [A, G, M, X, B, Q, T, F]。

    (calibrated_outputs[0] - calibrated_outputs[8])^2 + ...([7] - [15])^2
    Dont need ref_id，depends on the order.    """
    
    def __init__(self, distance_metric='l2_squared'):
        super(PairedDifferenceLoss, self).__init__()
        self.distance_metric = distance_metric
        if distance_metric not in ['l2_squared', 'l1']:
            raise ValueError("only support 'l2_squared' or 'l1' ")

    def forward(self, outputs, masks, zps, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):
        original_shape = outputs.shape
        batch_size = original_shape[0]
        
        if batch_size % 2 != 0:
            raise ValueError(f"PairedDifferenceLoss need batch_size == 2n, but received {batch_size}")
            
        if not (batch_size == zps.shape[0]):
             raise ValueError("outputs and zps must have the same batch_size")
            
        flux_outputs = flux_elliptical_jiefeng(outputs, masks, aperture_x, aperture_y,
                                   aperture_theta, aperture_a, aperture_b)
        calibrated_flux_outputs = flux_outputs * zps
        half_B = batch_size // 2
        
        outputs_1 = calibrated_outputs[0:half_B]#I usually set as 8
        outputs_2 = calibrated_outputs[half_B:]
        diff = outputs_1 - outputs_2

        if self.distance_metric == 'l2_squared':
            loss = (diff**2).mean() 
        elif self.distance_metric == 'l1':
            loss = torch.abs(diff).mean()
            
        return loss

In [70]:
class unbiasLoss(nn.Module):
    def __init__(self, distance_metric='l1'):
        super(unbiasLoss, self).__init__()
        self.distance_metric = distance_metric
        if distance_metric not in ['l2_squared', 'l1']:
            raise ValueError("only support 'l2_squared' or 'l1'")

    def forward(self, labels, outputs, masks, aperture_x, aperture_y, aperture_theta, aperture_a, aperture_b):
        original_shape = outputs.shape
        batch_size = original_shape[0]
                                
        total_loss = 0.0

        flux_outputs = flux_elliptical_jiefeng(outputs, masks, aperture_x, aperture_y,
                                   aperture_theta, aperture_a, aperture_b)
        flux_labels = flux_elliptical_jiefeng(labels, masks, aperture_x, aperture_y,
                                   aperture_theta, aperture_a, aperture_b)
        diff = labels - outputs
        
        if self.distance_metric == 'l2_squared':
            total_loss += (diff**2).sum()
        elif self.distance_metric == 'l1':
            total_loss += torch.abs(diff).sum()

        return total_loss 

In [71]:
#test the train
num_epochs = 1
lora_model.train()

loss_df = PairedDifferenceLoss(distance_metric='l2_squared')
loss_unbias = unbiasLoss(distance_metric='l1')

for epoch in range(num_epochs):
    for tensor_stamp_train, tensor_mask_train, features_df_train_tensor in traindataloader:
        
        inputs = tensor_stamp_train.to(device)
        masks = tensor_mask_train.to(device) 
        #features = features_df_train_tensor.to(device)
        #'ref_id', 'zp', 'aperture_x', 'aperture_y', 'aperture_theta', 'aperture_a', 'aperture_b'
        print(features[:, 0])
        optimizer.zero_grad()
        
        outputs = lora_model(inputs) 
        print(features[:,0])
        0/0
        loss1 = loss_unbias(inputs, outputs, masks, features[:,2],features[:,3],features[:,4],features[:,5],features[:,6])
        loss2 = loss_df(outputs, masks, features[:,0],features[:,1],features[:,2],features[:,3],features[:,4],features[:,5],features[:,6])
        loss = loss1 + loss2
        print(loss)
        0/0
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

tensor([ 99967.,  24674.,  88305.,  36282.,  51738.,  90629.,  14701.,  40444.,
         92580.,  49725.,  52638.,  26080.,  60232.,  27789.,  55907.,  25835.,
         98261.,  23923.,   9307.,  72193.,  79519.,  35163., 109869.,  51347.,
        112576.,  25576.,  90556.,  55209.,  85886.,  35554.,  52366.,  87143.],
       device='cuda:0')


OutOfMemoryError: CUDA out of memory. Tried to allocate 72.00 MiB. GPU 0 has a total capacity of 10.57 GiB of which 18.06 MiB is free. Process 3103597 has 4.66 GiB memory in use. Process 3141894 has 5.89 GiB memory in use. Of the allocated memory 5.69 GiB is allocated by PyTorch, and 20.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)